In [1]:
# === The basic libs
import numpy as np
np.random.seed(123)  # for reproducibility
import pandas as pd
from matplotlib import pyplot as plt
# === tf and keras libs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
import tensorflow.keras.utils as np_utils

In [2]:
# === Settings needed not to run out of GPU memory
print(tf.version.VERSION)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

2.0.0
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2060, pci bus id: 0000:08:00.0, compute capability: 7.5
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



In [3]:
import os
os.getenv('HOST_PIPE_DIR')

'/home/rishin/workspace/ffx-ai/dist/'

In [4]:
NUM_CLASSES = 30
#INPUT_SHAPE= (224, 224) # resnt, vgg, inception, mobilenet with default keras app take this only
INPUT_SHAPE= (299, 299)
BATCH_SIZE = 16
DIST_PATH = os.getenv('HOST_PIPE_DIR')

In [5]:
df_all = pd.read_csv(os.path.join(DIST_PATH, 'img_map.csv'))
df_all.head()

,Unnamed: 0,cls,id
0,0,helms_front,/home/rishin/workspace/ffx-ai/dist/mon_m001_04...
1,1,helms_front,/home/rishin/workspace/ffx-ai/dist/mon_m002_04...
2,2,helms_front,/home/rishin/workspace/ffx-ai/dist/mon_m003_04...
3,3,helms_front,/home/rishin/workspace/ffx-ai/dist/mon_m004_04...
4,4,helms_front,/home/rishin/workspace/ffx-ai/dist/mon_m005_04...


In [6]:
NUM_CLASSES = len(df_all.cls.unique())

In [7]:
def train_test_split(df, test_size=0.2, train_size=None):
    # determine train test split ratio
    df_len = len(df)
    test_size = test_size/df_len if isinstance(test_size, int) else test_size
    train_size = 1 - test_size if train_size is None else train_size
    ratio = train_size/df_len if isinstance(train_size, int) else train_size
    # split df
    msk = np.random.rand(df_len) < ratio
    train = df[msk]
    test = df[~msk]
    return train, test

In [8]:
df_train, df_test = train_test_split(df_all, test_size=NUM_CLASSES*20)

In [9]:
# https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c

In [10]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255.,
    rotation_range=22,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    
    validation_split=0.10
    )

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory=DIST_PATH,
    x_col="id",
    y_col="cls",
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode="categorical",
    target_size=INPUT_SHAPE)

valid_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory=DIST_PATH,
    x_col="id",
    y_col="cls",
    subset="validation",
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode="categorical",
    target_size=INPUT_SHAPE)

Found 19550 validated image filenames belonging to 30 classes.
Found 1955 validated image filenames belonging to 30 classes.


In [11]:
test_datagen= tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory=DIST_PATH,
    x_col="id",
    y_col="cls",
    batch_size=BATCH_SIZE,
    shuffle=False,
    target_size=INPUT_SHAPE)

Found 610 validated image filenames belonging to 30 classes.


In [12]:
#model = tf.keras.applications.mobilenet_v2.MobileNetV2(weights=None, classes=NUM_CLASSES)
model = tf.keras.applications.inception_v3.InceptionV3(weights=None, classes=NUM_CLASSES)

In [13]:
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"])

In [14]:
STEP_SIZE_TRAIN = train_generator.n / train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n /valid_generator.batch_size
STEP_SIZE_TEST = test_generator.n / test_generator.batch_size

In [15]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=40
)

Epoch 1/40
1222/1221 [==============================] - 593s 485ms/step - loss: 1.9665 - accuracy: 0.3628 - val_loss: 1.0902 - val_accuracy: 0.6061
Epoch 2/40
1222/1221 [==============================] - 580s 475ms/step - loss: 0.6262 - accuracy: 0.7562 - val_loss: 0.3354 - val_accuracy: 0.8542
Epoch 3/40
1222/1221 [==============================] - 590s 483ms/step - loss: 0.3503 - accuracy: 0.8619 - val_loss: 0.6095 - val_accuracy: 0.8256
Epoch 4/40
1222/1221 [==============================] - 585s 479ms/step - loss: 0.2531 - accuracy: 0.8981 - val_loss: 0.3624 - val_accuracy: 0.8373
Epoch 5/40
1222/1221 [==============================] - 589s 482ms/step - loss: 0.1955 - accuracy: 0.9167 - val_loss: 1.0213 - val_accuracy: 0.7540
Epoch 6/40
 232/1221 [====>.........................] - ETA: 7:11 - loss: 0.1705 - accuracy: 0.9302

KeyboardInterrupt: 

In [ ]:
dsfdsfdsf

test_generator.reset()
for i in test_generator:
    print(i.shape)
    break

In [ ]:
test_generator.reset()
y_pred = model.predict_generator(test_generator, steps=STEP_SIZE_TEST, verbose=1)

In [ ]:
predicted_class_indices = np.argmax(y_pred, axis=1)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
predictions